<a href="https://colab.research.google.com/github/qdcdescalzota-star/CPE-310-2526A/blob/main/DESCALZOTA_Midterm_Lab_Assessment_Data_Wrangling_with_Pandas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Exercise 1.
Read in the meteorite data from the Meteorite_Landings.csv file, rename the mass (g) column to mass, and drop all the latitude and longitude columns. Sort the result by mass in descending order.

In [22]:
import pandas as pd

# Read the CSV file
meteorites = pd.read_csv('Meteorite_Landings.csv')

# Rename the 'mass (g)' column to 'mass'
meteorites = meteorites.rename(columns={'mass (g)': 'mass'})

# Drop latitude and longitude columns
meteorites = meteorites.drop(columns=['reclat', 'reclong'], errors='ignore')

# Sort by mass in descending order
meteorites_sorted = meteorites.sort_values(by='mass', ascending=False)

print(meteorites_sorted.head())


                  name     id nametype      recclass        mass   fall  \
16392             Hoba  11890    Valid     Iron, IVB  60000000.0  Found   
5373         Cape York   5262    Valid   Iron, IIIAB  58200000.0  Found   
5365   Campo del Cielo   5247    Valid  Iron, IAB-MG  50000000.0  Found   
5370     Canyon Diablo   5257    Valid  Iron, IAB-MG  30000000.0  Found   
3455           Armanty   2335    Valid    Iron, IIIE  28000000.0  Found   

                         year             GeoLocation  
16392  01/01/1920 12:00:00 AM   (-19.58333, 17.91667)  
5373   01/01/1818 12:00:00 AM   (76.13333, -64.93333)  
5365   12/22/1575 12:00:00 AM  (-27.46667, -60.58333)  
5370   01/01/1891 12:00:00 AM     (35.05, -111.03333)  
3455   01/01/1898 12:00:00 AM            (47.0, 88.0)  


Exercise 2.
Using the meteorite data from the Meteorite_Landings.csv file, update the year column to only contain the year, convert it to a numeric data type, and create a new column indicating whether the meteorite was observed falling before 1970. Set the index to the id column and extract all the rows with IDs between 10,036 and 10,040 (inclusive) with loc[].

In [23]:
# Extracting only the year from the 'year' column
meteorites['year'] = meteorites['year'].astype(str).str.slice(0, 4)

# Convert to numeric
meteorites['year'] = pd.to_numeric(meteorites['year'], errors='coerce')

# New column for meteorites observed before 1970
meteorites['before_1970'] = meteorites['year'] < 1970

# Set 'id' as the index
meteorites = meteorites.set_index('id').sort_index()

# Extract rows with IDs between 10036 and 10040
subset = meteorites.loc[10036:10040]
print(subset)


            name nametype         recclass      mass   fall  year  \
id                                                                  
10036     Enigma    Valid               H4      94.0  Found   NaN   
10037       Enon    Valid  Iron, ungrouped     763.0  Found   NaN   
10038      Enshi    Valid               H5    8000.0   Fell   NaN   
10039  Ensisheim    Valid              LL6  127000.0   Fell   NaN   

                 GeoLocation  before_1970  
id                                         
10036  (31.33333, -82.31667)        False  
10037     (39.86667, -83.95)        False  
10038          (30.3, 109.5)        False  
10039       (47.86667, 7.35)        False  


Exercise 3.
A) Using the meteorite data from the Meteorite_Landings.csv file, create a pivot table that shows both the number of meteorites and the 95th percentile of meteorite mass for those that were found versus observed falling per year from 2005 through 2009 (inclusive). Hint: Be sure to convert the year column to a number as we did in the previous exercise.
B) Using the meteorite data from the Meteorite_Landings.csv file, compare summary statistics of the mass column for the meteorites that were found versus observed falling.

In [24]:
# Filter data between 2005 and 2009
meteorites_filtered = meteorites[(meteorites['year'] >= 2005) & (meteorites['year'] <= 2009)]

# pivot table
pivot = pd.pivot_table(
    meteorites_filtered,
    values='mass',
    index='year',
    columns='fall',
    aggfunc={'mass': ['count', lambda x: x.quantile(0.95)]}
)

print(pivot)


Empty DataFrame
Columns: []
Index: []


In [25]:

summary = meteorites.groupby('fall')['mass'].describe()
print(summary)


         count          mean            std  min     25%     50%      75%  \
fall                                                                        
Fell    1075.0  47070.715023  717067.125826  0.1  686.00  2800.0  10450.0   
Found  44510.0  12461.922983  571105.752311  0.0    6.94    30.5    178.0   

              max  
fall               
Fell   23000000.0  
Found  60000000.0  


Exercise 4.
Using the taxi trip data in the 2019_Yellow_Taxi_Trip_Data.csv file, resample the data to an hourly frequency based on the drop-off time. Calculate the total trip_distance, fare_amount, tolls_amount, and tip_amount, then find the 5 hours with the most tips.

In [26]:
# Read taxi data
taxi = pd.read_csv('2019_Yellow_Taxi_Trip_Data.csv', parse_dates=['tpep_dropoff_datetime'])

# Resample by hour based on drop-off time
taxi_hourly = (
    taxi.resample('h', on='tpep_dropoff_datetime')
        [['trip_distance', 'fare_amount', 'tolls_amount', 'tip_amount']]
        .sum()
)

# Find top 5 hours with most tips
top_tips = taxi_hourly.sort_values('tip_amount', ascending=False).head(5)
print(top_tips)


                       trip_distance  fare_amount  tolls_amount  tip_amount
tpep_dropoff_datetime                                                      
2019-10-23 16:00:00         10676.95     67797.76        699.04    12228.64
2019-10-23 17:00:00         16052.83     70131.91       4044.04    12044.03
2019-10-23 18:00:00          3104.56     11565.56       1454.67     1907.64
2019-10-23 15:00:00            14.34       213.50          0.00       51.75
2019-10-23 19:00:00            98.59       268.00         24.48       25.74
